In [21]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM, GRU
from keras.preprocessing import text
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from __future__ import print_function
from keras.layers.core import Activation, TimeDistributedDense, RepeatVector
from keras.layers import recurrent
import numpy as np

In [3]:
class CharacterTable(object):
    '''
    Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilties to their character output
    '''
    def __init__(self, chars, maxlen):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
        self.maxlen = maxlen

    def encode(self, C, maxlen=None):
        maxlen = maxlen if maxlen else self.maxlen
        X = np.zeros((maxlen, len(self.chars)))
        for i, c in enumerate(C):
            X[i, self.char_indices[c]] = 1
        return X

    def decode(self, X, calc_argmax=True):
        if calc_argmax:
            X = X.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in X)

In [7]:
def generate(digits):
    return ''.join([str(np.random.randint(0, 9)) for _ in range(digits)])

In [101]:
TRAINING_SIZE = 50000
TEST_SIZE = 10000
DIGITS = 50
MAXLEN = DIGITS
chars = '0123456789'
ctable = CharacterTable(chars, MAXLEN)

In [102]:
inputs = []
outputs = []
inputs_t = []
outputs_t = []
print('Generating data...')
while len(inputs) < TRAINING_SIZE:
    s = generate(DIGITS)
    inputs.append(s)
    outputs.append(s[::-1])
    
while len(inputs_t) < TEST_SIZE:
    s = generate(DIGITS)
    inputs_t.append(s)
    outputs_t.append(s[::-1])
print(inputs[12])
print(outputs[12])
print(inputs_t[12])
print(outputs_t[12])

Generating data...
12646768511134871277110632248837825024486363833255
55233836368442052873884223601177217843111586764621
81130105305183780231103662827473352420108774313008
80031347780102425337472826630113208738150350103118


In [103]:
print('Vectorization...')
X = np.zeros((len(inputs), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(outputs), MAXLEN, len(chars)), dtype=np.bool)
for i, sentence in enumerate(inputs):
    X[i] = ctable.encode(sentence, maxlen=MAXLEN)
for i, sentence in enumerate(outputs):
    y[i] = ctable.encode(sentence, maxlen=MAXLEN)
    
X_test = np.zeros((len(inputs_t), MAXLEN, len(chars)), dtype=np.bool)
y_test = np.zeros((len(outputs_t), MAXLEN, len(chars)), dtype=np.bool)
for i, sentence in enumerate(inputs_t):
    X_test[i] = ctable.encode(sentence, maxlen=MAXLEN)
for i, sentence in enumerate(outputs_t):
    y_test[i] = ctable.encode(sentence, maxlen=MAXLEN)
    
# print(X[12])

Vectorization...


In [104]:
indices = np.arange(len(y))
np.random.shuffle(indices)
X = X[indices]
y = y[indices]
print(X.shape)
print(y.shape)
print(X_test.shape)
print(y_test.shape)

(50000, 50, 10)
(50000, 50, 10)
(10000, 50, 10)
(10000, 50, 10)


In [113]:
HIDDEN_SIZE = 128
BATCH_SIZE = 200
LAYERS = 4

In [114]:
print('Build model...')
model = Sequential()
# dropout_W=0.1, dropout_U=0.1,
model.add(LSTM(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars)), return_sequences=True))
for _ in range(LAYERS - 2):
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True))
model.add(LSTM(HIDDEN_SIZE))
model.add(RepeatVector(MAXLEN))
for _ in range(LAYERS):
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True))

model.add(TimeDistributedDense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam')

Build model...


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
hist = model.fit(X, y, batch_size=BATCH_SIZE, nb_epoch=5, callbacks=[early_stopping],
          show_accuracy=True,validation_split = 0.1, shuffle=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/5
 9200/45000 [=====>........................] - ETA: 104s - loss: 0.1703 - acc: 0.9378

In [112]:
score, acc = model.evaluate(X_test, y_test,
                            batch_size=BATCH_SIZE,
                            show_accuracy=True)
print('Test score:', score)
print('Test accuracy:', acc)

10000/10000 [==============================] - 4s     
Test score: 0.171377762556
Test accuracy: 0.940928


In [70]:
test = '12345678901234567898'
x_test = ctable.encode(test, maxlen=MAXLEN)
X_test = np.zeros((1, MAXLEN, len(chars)), dtype=np.bool)
X_test[0] = x_test
res = model.predict_classes(X_test)
res[0]

1/1 [==============================] - 0s


array([8, 8, 7, 7, 6, 5, 4, 3, 2, 1, 0, 0, 8, 7, 7, 5, 4, 3, 2, 1])

In [28]:
import matplotlib.pyplot as plt
axes = plt.gca()
x_min = hist.epoch[0]
x_max = hist.epoch[-1]+1
axes.set_xlim([x_min,x_max])

plt.scatter(hist.epoch, hist.history['loss'], color='g')
plt.plot(hist.history['loss'], color='g', label='Training Loss')
plt.scatter(hist.epoch, hist.history['val_loss'], color='b')
plt.plot(hist.history['val_loss'], color='b', label='Validation Loss')
plt.xlabel('epochs')
plt.ylabel('Loss')
plt.title('Training Loss & Validation Loss vs Epochs')
plt.legend()

plt.figure(2)

axes = plt.gca()
x_min = hist.epoch[0]
x_max = hist.epoch[-1]+1
axes.set_xlim([x_min,x_max])

plt.scatter(hist.epoch, hist.history['acc'], color='r')
plt.plot(hist.history['acc'], color='r', label='Training Accuracy')
plt.scatter(hist.epoch, hist.history['val_acc'], color='c')
plt.plot(hist.history['val_acc'], color='c', label='Validation Accuracy')
plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.title('Trainging Accuracy & Validation Accuracy vs Epochs')
plt.legend()

plt.show()